# Aula 04 - Windows Function

Documentação Postgres:  https://www.postgresql.org/docs/current/functions-window.html

In [1]:
import psycopg
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Conectar ao banco de dados PostgreSQL
conn = psycopg.connect("dbname=postgres user=postgres password=password host=172.25.224.1 port=5432")
cursor = conn.cursor()

## Com o que vimos até aqui: Group By

Com SQL que vimos até agora conseguimos dois tipos de resultado: todas as linhas (com ou sem filtro/where) ou linhas agrupadas (group by)

Cálcular:
* Quantos produtos únicos existem?
* Quantos produtos no total?
* Qual é o valor total pago?

In [3]:
query = '''
SELECT order_id,
       COUNT(order_id) AS unique_product,
       SUM(quantity) AS total_quantity,
       SUM(unit_price * quantity) AS total_price
FROM order_details
GROUP BY order_id
ORDER BY order_id;
'''
pd.read_sql(query, conn)

,order_id,unique_product,total_quantity,total_price
0,10248,3,27,439.999998
1,10249,2,49,1863.400064
2,10250,3,60,1813.000040
3,10251,3,41,670.799986
4,10252,3,105,3730.000153
...,...,...,...,...
825,11073,2,30,300.000000
826,11074,1,14,244.300011
827,11075,3,42,586.000000
828,11076,3,50,1056.999998


## Com Windows Function

As `Windows Function` permitem uma análise de dados eficiente e precisa, ao possibilitar cálculos dentro de `partições ou linhas específicas`. Elas são cruciais para tarefas como classificação, agregação e análise de tendências em consultas SQL.

Essas funções são aplicadas a cada linha de um conjunto de resultados, e utilizam uma cláusula `OVER()` para determinar como cada linha é processada dentro de uma "janela", permitindo controle sobre o comportamento da função dentro de um grupo de dados ordenados.

Window Functions Syntax componentes
```sql
window_function_name(arg1, arg2, ...) OVER (
  [PARTITION BY partition_expression, ...]
  [ORDER BY sort_expression [ASC | DESC], ...]
)
```

* **window_function_name**: Este é o nome da função de janela que você deseja usar, como SUM, RANK, LEAD, etc.

* **arg1, arg2, ...:** Estes são os argumentos que você passa para a função de janela, se ela exigir algum. Por exemplo, para a função SUM, você especificaria a coluna que deseja somar.

* **OVER**: Principal conceito das windows functions, ele que cria essa "Janela" onde fazem nossos cálculos

* **PARTITION BY:** Esta cláusula opcional divide o conjunto de resultados em partições ou grupos. A função de janela opera independentemente dentro de cada partição.

* **ORDER BY:** Esta cláusula opcional especifica a ordem em que as linhas são processadas dentro de cada partição. Você pode especificar a ordem ascendente (ASC) ou descendente (DESC).

In [5]:
query = '''
SELECT DISTINCT order_id,
   COUNT(order_id) OVER (PARTITION BY order_id) AS unique_product,
   SUM(quantity) OVER (PARTITION BY order_id) AS total_quantity,
   SUM(unit_price * quantity) OVER (PARTITION BY order_id) AS total_price
FROM order_details
ORDER BY order_id;
'''
pd.read_sql(query, conn)


,order_id,unique_product,total_quantity,total_price
0,10248,3,27,439.999998
1,10249,2,49,1863.400064
2,10250,3,60,1813.000040
3,10251,3,41,670.799986
4,10252,3,105,3730.000153
...,...,...,...,...
825,11073,2,30,300.000000
826,11074,1,14,244.300011
827,11075,3,42,586.000000
828,11076,3,50,1056.999998


## MIN (), MAX (), AVG ()

Quais são os valores mínimo, máximo e médio de frete pago por cada cliente? (tabela orders)

### Usando Group by

In [6]:
query = '''
SELECT customer_id,
   MIN(freight) AS min_freight,
   MAX(freight) AS max_freight,
   AVG(freight) AS avg_freight
FROM orders
GROUP BY customer_id
ORDER BY customer_id;
''' 
pd.read_sql(query, conn)

,customer_id,min_freight,max_freight,avg_freight
0,ALFKI,1.21,69.53,37.596666
1,ANATR,1.61,43.90,24.355000
2,ANTON,4.03,84.84,38.360000
3,AROUT,3.04,146.32,36.303847
4,BERGS,3.50,244.79,86.640000
...,...,...,...,...
84,WARTH,0.59,180.45,54.832000
85,WELLI,0.14,55.23,21.634444
86,WHITC,4.56,606.19,96.647143
87,WILMK,0.75,38.11,12.630000


### Detalhes da Consulta Ajustada:

* **`customer_id`**: Seleciona o identificador único do cliente da tabela `orders`.
* **`MIN(freight) AS min_freight`**: Calcula o valor mínimo de frete para cada cliente.
* **`MAX(freight) AS max_freight`**: Calcula o valor máximo de frete para cada cliente.
* **`AVG(freight) AS avg_freight`**: Calcula o valor médio de frete para cada cliente.

### Explicação:

* A função `MIN` extrai o menor valor de frete registrado para cada cliente.
* A função `MAX` obtém o maior valor de frete registrado para cada cliente.
* A função `AVG` fornece o valor médio de frete por cliente, útil para entender o custo médio de envio associado a cada um.
* `GROUP BY customer_id` agrupa os registros por `customer_id`, permitindo que as funções agregadas calculem seus resultados para cada grupo de cliente.
* `ORDER BY customer_id` garante que os resultados sejam apresentados em ordem crescente de `customer_id`, facilitando a leitura e a análise dos dados.

### Usando Windows Function

In [7]:
query = ''' 
SELECT DISTINCT customer_id,
   MIN(freight) OVER (PARTITION BY customer_id) AS min_freight,
   MAX(freight) OVER (PARTITION BY customer_id) AS max_freight,
   AVG(freight) OVER (PARTITION BY customer_id) AS avg_freight
FROM orders
ORDER BY customer_id;
''' 
pd.read_sql(query, conn)

,customer_id,min_freight,max_freight,avg_freight
0,ALFKI,1.21,69.53,37.596666
1,ANATR,1.61,43.90,24.355000
2,ANTON,4.03,84.84,38.360000
3,AROUT,3.04,146.32,36.303847
4,BERGS,3.50,244.79,86.640000
...,...,...,...,...
84,WARTH,0.59,180.45,54.832000
85,WELLI,0.14,55.23,21.634444
86,WHITC,4.56,606.19,96.647143
87,WILMK,0.75,38.11,12.630000


### Explicação da Consulta Ajustada:

* **`customer_id`**: Seleciona o identificador único do cliente da tabela `orders`.
* **`MIN(freight) OVER (PARTITION BY customer_id)`**: Utiliza a função de janela `MIN` para calcular o valor mínimo de frete para cada grupo de registros que têm o mesmo `customer_id`.
* **`MAX(freight) OVER (PARTITION BY customer_id)`**: Utiliza a função de janela `MAX` para calcular o valor máximo de frete para cada `customer_id`.
* **`AVG(freight) OVER (PARTITION BY customer_id)`**: Utiliza a função de janela `AVG` para calcular o valor médio de frete para cada `customer_id`.

### Características das Funções de Janela:

* **Funções de Janela (`OVER`)**: As funções de janela permitem que você execute cálculos sobre um conjunto de linhas relacionadas a cada entrada. Ao usar o `PARTITION BY customer_id`, a função de janela é reiniciada para cada novo `customer_id`. Isso significa que cada cálculo de `MIN`, `MAX`, e `AVG` é confinado ao conjunto de ordens de cada cliente individualmente.
* **`DISTINCT`**: A cláusula `DISTINCT` é utilizada para garantir que cada `customer_id` apareça apenas uma vez nos resultados finais, juntamente com seus respectivos valores de frete mínimo, máximo e médio. Isso é necessário porque as funções de janela calculam valores para cada linha, e sem `DISTINCT`, cada `customer_id` poderia aparecer múltiplas vezes se houver várias ordens por cliente.

## Colapso

Para ilustrar como a cláusula `GROUP BY` influencia os resultados de uma consulta SQL e por que ela pode "colapsar" as linhas para uma única linha por grupo, vou dar um exemplo baseado nas funções de agregação `MIN`, `MAX`, e `AVG` que discutimos anteriormente. Essas funções são frequentemente usadas para calcular estatísticas resumidas dentro de cada grupo especificado por `GROUP BY`.

### Exemplo sem GROUP BY

Considere a seguinte consulta sem usar `GROUP BY`:

In [8]:
query = ''' 
-- 830 linhas
SELECT customer_id, freight
FROM orders;
'''
pd.read_sql(query, conn)

,customer_id,freight
0,VINET,32.38
1,TOMSP,11.61
2,HANAR,65.83
3,VICTE,41.34
4,SUPRD,51.30
...,...,...
825,PERIC,24.95
826,SIMOB,18.44
827,RICSU,6.19
828,BONAP,38.28


Essa consulta simplesmente seleciona o `customer_id` e o `freight` de cada ordem. Se houver múltiplas ordens para cada cliente, cada ordem aparecerá como uma linha separada no conjunto de resultados.

### Exemplo com GROUP BY

Agora, vamos adicionar `GROUP BY` e funções de agregação:

In [13]:
query = ''' 
-- 89 linhas
-- EXPLAIN ANALYZE //informa to custo de cada operação na ordem e o tempo total
SELECT customer_id,
       MIN(freight) AS min_freight,
       MAX(freight) AS max_freight,
       AVG(freight) AS avg_freight
FROM orders
GROUP BY customer_id
ORDER BY customer_id;
''' 
pd.read_sql(query, conn)

,customer_id,min_freight,max_freight,avg_freight
0,ALFKI,1.21,69.53,37.596666
1,ANATR,1.61,43.90,24.355000
2,ANTON,4.03,84.84,38.360000
3,AROUT,3.04,146.32,36.303847
4,BERGS,3.50,244.79,86.640000
...,...,...,...,...
84,WARTH,0.59,180.45,54.832000
85,WELLI,0.14,55.23,21.634444
86,WHITC,4.56,606.19,96.647143
87,WILMK,0.75,38.11,12.630000


### O que acontece aqui:

* **`GROUP BY customer_id`**: Esta cláusula agrupa todas as entradas na tabela `orders` que têm o mesmo `customer_id`. Para cada grupo, a consulta calcula os valores mínimo, máximo e médio de `freight`.
    
* **Agregações (`MIN`, `MAX`, `AVG`)**: Cada uma dessas funções de agregação opera sobre o conjunto de `freight` dentro do grupo especificado pelo `customer_id`. Apenas um valor para cada função de agregação é retornado por grupo.

### Por que "colapsa" as linhas:

* Quando usamos `GROUP BY`, a consulta não retorna mais uma linha para cada entrada na tabela `orders`. Em vez disso, ela retorna uma linha para cada grupo de `customer_id`, onde cada linha contém o `customer_id` e os valores agregados de `freight` para esse grupo. Isso significa que se um cliente tem várias ordens, você não verá cada ordem individualmente; em vez disso, você verá uma linha resumida com as estatísticas de frete para todas as ordens desse cliente.

### Limitação do SELECT com GROUP BY:

* Se você tentar selecionar uma coluna que não está incluída na cláusula `GROUP BY` e que não é uma expressão agregada, a consulta falhará. Por exemplo, a consulta a seguir resultará em erro porque `order_date` não está em uma função agregada nem no `GROUP BY`:

In [15]:
query = ''' 
SELECT customer_id, order_date, AVG(freight) AS avg_freight
FROM orders
GROUP BY customer_id, order_date;
'''
pd.read_sql(query, conn)

,customer_id,order_date,avg_freight
0,LILAS,1997-12-16,42.130001
1,RICAR,1998-04-29,85.800003
2,ANTON,1997-09-25,36.130001
3,GODOS,1996-09-11,107.830002
4,THEBI,1997-12-30,237.339996
...,...,...,...
818,BOTTM,1998-03-27,14.010000
819,VAFFE,1998-03-12,27.200001
820,VICTE,1998-01-05,130.940002
821,DRACD,1996-12-23,5.450000


### Mensagem de Erro Típica:

* Em muitos sistemas de gerenciamento de banco de dados, como PostgreSQL ou MySQL, essa consulta resultaria em um erro como: "column "orders.order_date" must appear in the GROUP BY clause or be used in an aggregate function".

Este exemplo mostra claramente como o `GROUP BY` "colapsa" as linhas em grupos, permitindo cálculos resumidos, mas também impõe restrições sobre quais colunas podem ser selecionadas diretamente.

Para ilustrar como evitar o "colapso" das linhas utilizando funções de janela (window functions) em vez de `GROUP BY`, vamos utilizar as mesmas estatísticas de frete (mínimo, máximo e médio) por cliente, mas manter todas as linhas de pedidos individuais visíveis no conjunto de resultados. As funções de janela permitem calcular agregações enquanto ainda se mantém cada linha distinta na saída.

### Consulta com Funções de Janela

Aqui está como você pode escrever uma consulta que utiliza funções de janela para calcular o frete mínimo, máximo e médio para cada cliente sem colapsar as linhas:

In [16]:
query = ''' 
SELECT 
    customer_id,
    order_id,  -- Mantendo a visibilidade de cada pedido
    freight,
    MIN(freight) OVER (PARTITION BY customer_id) AS min_freight,
    MAX(freight) OVER (PARTITION BY customer_id) AS max_freight,
    AVG(freight) OVER (PARTITION BY customer_id) AS avg_freight
FROM orders
ORDER BY customer_id, order_id;
'''
pd.read_sql(query, conn)

,customer_id,order_id,freight,min_freight,max_freight,avg_freight
0,ALFKI,10643,29.46,1.21,69.53,37.596666
1,ALFKI,10692,61.02,1.21,69.53,37.596666
2,ALFKI,10702,23.94,1.21,69.53,37.596666
3,ALFKI,10835,69.53,1.21,69.53,37.596666
4,ALFKI,10952,40.42,1.21,69.53,37.596666
...,...,...,...,...,...,...
825,WOLZA,10792,23.79,3.94,80.65,25.105715
826,WOLZA,10870,12.04,3.94,80.65,25.105715
827,WOLZA,10906,26.29,3.94,80.65,25.105715
828,WOLZA,10998,20.31,3.94,80.65,25.105715


### Explicação da Consulta

* **Seleção de Colunas**: `customer_id`, `order_id`, e `freight` são selecionados diretamente, o que mantém cada linha de pedido individual visível no resultado.
* **Funções de Janela**: `MIN(freight) OVER`, `MAX(freight) OVER`, e `AVG(freight) OVER` são aplicadas com a cláusula `PARTITION BY customer_id`. Isso significa que as estatísticas de frete são calculadas para cada grupo de `customer_id`, mas a aplicação é feita sem agrupar as linhas em um único resultado por cliente. Cada linha no conjunto de resultados original mantém sua identidade única.
* **`PARTITION BY customer_id`**: Assegura que as funções de janela são recalculadas para cada cliente. Cada pedido mantém sua linha, mas agora também inclui as informações agregadas de frete específicas para o cliente ao qual o pedido pertence.
* **`ORDER BY customer_id, order_id`**: Ordena os resultados primeiro por `customer_id` e depois por `order_id`, facilitando a leitura dos dados.

### Vantagens das Funções de Janela

* **Preservação de Dados Detalhados**: Ao contrário do `GROUP BY`, que agrega e reduz os dados a uma linha por grupo, as funções de janela mantêm cada linha individual do conjunto de dados original visível. Isso é útil para análises detalhadas onde você precisa ver tanto os valores agregados quanto os dados de linha individual.
* **Flexibilidade**: Você pode calcular múltiplas métricas de agregação em diferentes partições dentro da mesma consulta sem múltiplas passagens pelos dados ou subconsultas complexas.

Este método é especialmente útil em relatórios e análises detalhadas onde tanto o contexto agregado quanto os detalhes de cada evento individual (neste caso, cada pedido) são importantes para uma compreensão completa dos dados.

Funções de classificação de janela no SQL são um conjunto de ferramentas valiosas usadas para atribuir classificações, posições ou números sequenciais às linhas dentro de um conjunto de resultados com base em critérios específicos.

Elas são aplicadas em vários cenários, como criar leaderboards, classificar produtos por vendas, identificar os melhores desempenhos ou acompanhar mudanças ao longo do tempo. Essas funções são ferramentas poderosas para obter insights e tomar decisões informadas na análise de dados.

### 2.1 RANK(), DENSE_RANK() e ROW_NUMBER()

* **RANK()**: Atribui um rank único a cada linha, deixando lacunas em caso de empates.
* **DENSE_RANK()**: Atribui um rank único a cada linha, com ranks contínuos para linhas empatadas.
* **ROW_NUMBER()**: Atribui um número inteiro sequencial único a cada linha, independentemente de empates, sem lacunas.

### Exemplo: Classificação dos produtos mais venvidos POR order ID

ex: o mesmo produto pode ficar em primeiro por ter vendido muito por ORDER e depois ficar em segundo por ter vendido muito por ORDER

In [17]:
query = ''' 
SELECT  
  o.order_id, 
  p.product_name, 
  (o.unit_price * o.quantity) AS total_sale,
  ROW_NUMBER() OVER (ORDER BY (o.unit_price * o.quantity) DESC) AS order_rn, 
  RANK() OVER (ORDER BY (o.unit_price * o.quantity) DESC) AS order_rank, 
  DENSE_RANK() OVER (ORDER BY (o.unit_price * o.quantity) DESC) AS order_dense
FROM  
  order_details o
JOIN 
  products p ON p.product_id = o.product_id;
'''
pd.read_sql(query, conn)

,order_id,product_name,total_sale,order_rn,order_rank,order_dense
0,10865,Côte de Blaye,15810.000000,1,1,1
1,10981,Côte de Blaye,15810.000000,2,1,1
2,10353,Côte de Blaye,10540.000153,3,3,2
3,10417,Côte de Blaye,10540.000153,4,3,2
4,10889,Côte de Blaye,10540.000000,5,5,3
...,...,...,...,...,...,...
2150,10782,Gorgonzola Telino,12.500000,2151,2151,966
2151,10850,Geitost,10.000000,2152,2152,967
2152,10420,Konbu,9.600000,2153,2153,968
2153,10281,Teatime Chocolate Biscuits,7.300000,2154,2154,969


### Explicação da Consulta

* **Seleção de Dados**: A consulta seleciona o `order_id`, `product_name` da tabela `products`, e calcula `total_sale` como o produto de `unit_price` e `quantity` da tabela `order_details`.
    
* **Funções de Classificação**:
    
    * **`ROW_NUMBER()`**: Atribui um número sequencial a cada linha baseada no total de vendas (`total_sale`), ordenado do maior para o menor. Cada linha recebe um número único dentro do conjunto de resultados inteiro.
    * **`RANK()`**: Atribui um rank a cada linha baseado no `total_sale`, onde linhas com valores iguais recebem o mesmo rank, e o próximo rank disponível considera os empates (por exemplo, se dois itens compartilham o primeiro lugar, o próximo item será o terceiro).
    * **`DENSE_RANK()`**: Funciona de forma similar ao `RANK()`, mas os ranks subsequentes não têm lacunas. Se dois itens estão empatados no primeiro lugar, o próximo item será o segundo.
* **`JOIN`**: A junção entre `order_details` e `products` é feita pelo `product_id`, permitindo que o nome do produto seja incluído nos resultados baseados nos IDs correspondentes em ambas as tabelas.

## Este relatório apresenta o ID de cada pedido juntamente com o total de vendas e a classificação percentual e a distribuição cumulativa do valor de cada venda em relação ao valor total das vendas para o mesmo pedido. Esses cálculos são realizados com base no preço unitário e na quantidade de produtos vendidos em cada pedido.

### Exemplo: Classificação dos produtos mais venvidos usando SUB QUERY

In [18]:
query = ''' 
SELECT  
  sales.product_name, 
  total_sale,
  ROW_NUMBER() OVER (ORDER BY total_sale DESC) AS order_rn, 
  RANK() OVER (ORDER BY total_sale DESC) AS order_rank, 
  DENSE_RANK() OVER (ORDER BY total_sale DESC) AS order_dense
FROM (
  SELECT 
    p.product_name, 
    SUM(o.unit_price * o.quantity) AS total_sale
  FROM  
    order_details o
  JOIN 
    products p ON p.product_id = o.product_id
  GROUP BY p.product_name
) AS sales
ORDER BY sales.product_name;
'''
pd.read_sql(query, conn)

,product_name,total_sale,order_rn,order_rank,order_dense
0,Alice Mutton,35482.200260,8,8,8
1,Aniseed Syrup,3080.000000,71,71,71
2,Boston Crab Meat,19048.299644,22,22,22
3,Camembert Pierrot,50286.000374,4,4,4
4,Carnarvon Tigers,31987.500000,9,9,9
...,...,...,...,...,...
72,Uncle Bob's Organic Dried Pears,22464.000000,16,16,16
73,Valkoinen suklaa,3510.000000,69,69,69
74,Vegie-spread,17696.300041,25,25,25
75,Wimmers gute Semmelknödel,23009.000092,15,15,15


### Utilidade da Consulta

Esta consulta é útil para análises de vendas, onde é necessário identificar os produtos mais vendidos, bem como sua classificação em termos de receita gerada. Ela permite que os analistas vejam rapidamente quais produtos geram mais receita e como eles se classificam em relação uns aos outros, facilitando decisões estratégicas relacionadas a estoque, promoções e planejamento de vendas.

### Funções PERCENT_RANK() e CUME_DIST()

Ambos retornam um valor entre 0 e 1

* **PERCENT_RANK()**: Calcula o rank relativo de uma linha específica dentro do conjunto de resultados como uma porcentagem. É computado usando a seguinte fórmula:
    * RANK é o rank da linha dentro do conjunto de resultados.
    * N é o número total de linhas no conjunto de resultados.
    * PERCENT_RANK = (RANK - 1) / (N - 1)
* **CUME_DIST()**: Calcula a distribuição acumulada de um valor no conjunto de resultados. Representa a proporção de linhas que são menores ou iguais à linha atual. A fórmula é a seguinte:
    * CUME_DIST = (Número de linhas com valores <= linha atual) / (Número total de linhas)

Ambas as funções PERCENT_RANK() e CUME_DIST() são valiosas para entender a distribuição e posição de pontos de dados dentro de um conjunto de dados, particularmente em cenários onde você deseja comparar a posição de um valor específico com a distribuição geral de dados.

In [19]:
query = ''' 
SELECT  
  order_id, 
  unit_price * quantity AS total_sale,
  ROUND(CAST(PERCENT_RANK() OVER (PARTITION BY order_id 
    ORDER BY (unit_price * quantity) DESC) AS numeric), 2) AS order_percent_rank,
  ROUND(CAST(CUME_DIST() OVER (PARTITION BY order_id 
    ORDER BY (unit_price * quantity) DESC) AS numeric), 2) AS order_cume_dist
FROM  
  order_details;
'''
pd.read_sql(query, conn)

,order_id,total_sale,order_percent_rank,order_cume_dist
0,10248,173.999996,0.00,0.33
1,10248,168.000000,0.50,0.67
2,10248,98.000002,1.00,1.00
3,10249,1696.000061,0.00,0.50
4,10249,167.400003,1.00,1.00
...,...,...,...,...
2150,11077,23.250000,0.83,0.84
2151,11077,22.000000,0.88,0.88
2152,11077,18.000000,0.92,0.92
2153,11077,17.000000,0.96,0.96


### Explicação da Consulta Ajustada:

* **Seleção de Dados**: A consulta seleciona o `order_id` e calcula `total_sale` como o produto de `unit_price` e `quantity`.
* **Funções de Janela**:
    * **`PERCENT_RANK()`**: Aplicada com uma partição por `order_id` e ordenada pelo `total_sale` de forma descendente, calcula a posição percentual de cada venda em relação a todas as outras no mesmo pedido.
    * **`CUME_DIST()`**: Similarmente, calcula a distribuição acumulada das vendas, indicando a proporção de vendas que não excedem o `total_sale` da linha atual dentro de cada pedido.
* **Arredondamento**: Os resultados de `PERCENT_RANK()` e `CUME_DIST()` são arredondados para duas casas decimais para facilitar a interpretação.

Esta consulta é útil para análises detalhadas de desempenho de vendas dentro de pedidos, permitindo que gestores e analistas identifiquem rapidamente quais itens contribuem mais

A função NTILE() no SQL é usada para dividir o conjunto de resultados em um número especificado de partes aproximadamente iguais ou "faixas" e atribuir um número de grupo ou "bucket" a cada linha com base em sua posição dentro do conjunto de resultados ordenado.
```sql
NTILE(n) OVER (ORDER BY coluna)
```

* **n**: O número de faixas ou grupos que você deseja criar.
* **ORDER BY coluna**: A coluna pela qual você deseja ordenar o conjunto de resultados antes de aplicar a função NTILE().

### Exemplo: Listar funcionários dividindo-os em 3 grupos


In [21]:
query = '''
SELECT first_name, last_name, title,
   NTILE(3) OVER (ORDER BY first_name) AS group_number
FROM employees;
''' 
pd.read_sql(query, conn)

,first_name,last_name,title,group_number
0,Andrew,Fuller,"Vice President, Sales",1
1,Anne,Dodsworth,Sales Representative,1
2,Janet,Leverling,Sales Representative,1
3,Laura,Callahan,Inside Sales Coordinator,2
4,Margaret,Peacock,Sales Representative,2
5,Michael,Suyama,Sales Representative,2
6,Nancy,Davolio,Sales Representative,3
7,Robert,King,Sales Representative,3
8,Steven,Buchanan,Sales Manager,3


### Explicação da Consulta Ajustada:

* **Seleção de Dados**: A consulta seleciona `first_name`, `last_name` e `title` da tabela `employees`.
* **NTILE(3) OVER (ORDER BY first_name)**: Aplica a função NTILE para dividir os funcionários em 3 grupos baseados na ordem alfabética de seus primeiros nomes. Cada funcionário receberá um número de grupo (`group_number`) que indica a qual dos três grupos ele pertence.

Esta consulta é útil para análises que requerem a distribuição equitativa dos dados em grupos especificados, como para balanceamento de cargas de trabalho, análises segmentadas, ou mesmo para fins de relatórios onde a divisão em grupos facilita a visualização e o entendimento dos dados.

LAG(), LEAD()

* **LAG()**: Permite acessar o valor da linha anterior dentro de um conjunto de resultados. Isso é particularmente útil para fazer comparações com a linha atual ou identificar tendências ao longo do tempo.
* **LEAD()**: Permite acessar o valor da próxima linha dentro de um conjunto de resultados, possibilitando comparações com a linha subsequente.

### Exemplo: Ordenando os custos de envio pagos pelos clientes de acordo com suas datas de pedido:

In [22]:
query = ''' 
SELECT 
  customer_id, 
  TO_CHAR(order_date, 'YYYY-MM-DD') AS order_date, 
  shippers.company_name AS shipper_name, 
  LAG(freight) OVER (PARTITION BY customer_id ORDER BY order_date DESC) AS previous_order_freight, 
  freight AS order_freight, 
  LEAD(freight) OVER (PARTITION BY customer_id ORDER BY order_date DESC) AS next_order_freight
FROM 
  orders
JOIN 
  shippers ON shippers.shipper_id = orders.ship_via;
''' 
pd.read_sql(query, conn)

,customer_id,order_date,shipper_name,previous_order_freight,order_freight,next_order_freight
0,ALFKI,1998-04-09,Speedy Express,NaN,1.21,40.42
1,ALFKI,1998-03-16,Speedy Express,1.21,40.42,69.53
2,ALFKI,1998-01-15,Federal Shipping,40.42,69.53,23.94
3,ALFKI,1997-10-13,Speedy Express,69.53,23.94,61.02
4,ALFKI,1997-10-03,United Package,23.94,61.02,29.46
...,...,...,...,...,...,...
825,WOLZA,1998-02-25,Federal Shipping,20.31,26.29,12.04
826,WOLZA,1998-02-04,Federal Shipping,26.29,12.04,23.79
827,WOLZA,1997-12-23,Federal Shipping,12.04,23.79,80.65
828,WOLZA,1997-07-25,United Package,23.79,80.65,3.94


* **LEAD() e LAG(): Estas funções de janela são usadas para acessar dados de linhas anteriores ou subsequentes dentro de uma partição definida, muito úteis para comparar o valor de frete entre ordens consecutivas de um mesmo cliente.**

# Desafio:

* Faça a classificação dos produtos mais venvidos usando usando RANK(), DENSE_RANK() e ROW_NUMBER()
* Essa questão tem 2 implementações, veja uma que utiliza subquery e uma que não utiliza.
* Tabelas utilizadasFROM order_details o JOIN products p ON p.product_id = o.product_id;

In [23]:
query = ''' 
-- Classificação de produtos mais vendidos com subquery (RANK)
WITH ProductSales AS (
    SELECT
        p.product_name,
        SUM(o.quantity) AS total_sold
    FROM
        order_details o
    JOIN
        products p ON p.product_id = o.product_id
    GROUP BY
        p.product_name
)
SELECT
    product_name,
    total_sold,
    RANK() OVER (ORDER BY total_sold DESC) AS rank_product
FROM
    ProductSales;
''' 
pd.read_sql(query, conn)


,product_name,total_sold,rank_product
0,Camembert Pierrot,1577,1
1,Raclette Courdavault,1496,2
2,Gorgonzola Telino,1397,3
3,Gnocchi di nonna Alice,1263,4
4,Pavlova,1158,5
...,...,...,...
72,Laughing Lumberjack Lager,184,73
73,Chocolade,138,74
74,Gravad lax,125,75
75,Genen Shouyu,122,76


In [24]:
query = ''' 
-- Classificação de produtos mais vendidos com subquery (DENSE_RANK)
WITH ProductSales AS (
    SELECT
        p.product_name,
        SUM(o.quantity) AS total_sold
    FROM
        order_details o
    JOIN
        products p ON p.product_id = o.product_id
    GROUP BY
        p.product_name
)
SELECT
    product_name,
    total_sold,
    DENSE_RANK() OVER (ORDER BY total_sold DESC) AS dense_rank_product
FROM
    ProductSales;
''' 
pd.read_sql(query, conn)

,product_name,total_sold,dense_rank_product
0,Camembert Pierrot,1577,1
1,Raclette Courdavault,1496,2
2,Gorgonzola Telino,1397,3
3,Gnocchi di nonna Alice,1263,4
4,Pavlova,1158,5
...,...,...,...
72,Laughing Lumberjack Lager,184,68
73,Chocolade,138,69
74,Gravad lax,125,70
75,Genen Shouyu,122,71


In [25]:
query = ''' 
-- Classificação de produtos mais vendidos com subquery (ROW_NUMBER)
WITH ProductSales AS (
    SELECT
        p.product_name,
        SUM(o.quantity) AS total_sold
    FROM
        order_details o
    JOIN
        products p ON p.product_id = o.product_id
    GROUP BY
        p.product_name
)
SELECT
    product_name,
    total_sold,
    ROW_NUMBER() OVER (ORDER BY total_sold DESC) AS row_number_product
FROM
    ProductSales;
'''
pd.read_sql(query, conn)

,product_name,total_sold,row_number_product
0,Camembert Pierrot,1577,1
1,Raclette Courdavault,1496,2
2,Gorgonzola Telino,1397,3
3,Gnocchi di nonna Alice,1263,4
4,Pavlova,1158,5
...,...,...,...
72,Laughing Lumberjack Lager,184,73
73,Chocolade,138,74
74,Gravad lax,125,75
75,Genen Shouyu,122,76


In [26]:
query = ''' 
-- Classificação de produtos mais vendidos sem subquery (RANK)
SELECT
    p.product_name,
    SUM(o.quantity) AS total_sold,
    RANK() OVER (ORDER BY SUM(o.quantity) DESC) AS rank_product
FROM
    order_details o
JOIN
    products p ON p.product_id = o.product_id
GROUP BY
    p.product_name;
'''
pd.read_sql(query, conn)

,product_name,total_sold,rank_product
0,Camembert Pierrot,1577,1
1,Raclette Courdavault,1496,2
2,Gorgonzola Telino,1397,3
3,Gnocchi di nonna Alice,1263,4
4,Pavlova,1158,5
...,...,...,...
72,Laughing Lumberjack Lager,184,73
73,Chocolade,138,74
74,Gravad lax,125,75
75,Genen Shouyu,122,76


In [27]:
query = ''' 
-- Classificação de produtos mais vendidos sem subquery (DENSE_RANK)
SELECT
    p.product_name,
    SUM(o.quantity) AS total_sold,
    DENSE_RANK() OVER (ORDER BY SUM(o.quantity) DESC) AS dense_rank_product
FROM
    order_details o
JOIN
    products p ON p.product_id = o.product_id
GROUP BY
    p.product_name;
'''
pd.read_sql(query, conn)

,product_name,total_sold,dense_rank_product
0,Camembert Pierrot,1577,1
1,Raclette Courdavault,1496,2
2,Gorgonzola Telino,1397,3
3,Gnocchi di nonna Alice,1263,4
4,Pavlova,1158,5
...,...,...,...
72,Laughing Lumberjack Lager,184,68
73,Chocolade,138,69
74,Gravad lax,125,70
75,Genen Shouyu,122,71


In [28]:
query = ''' 
-- Classificação de produtos mais vendidos sem subquery (ROW_NUMBER)
SELECT
    p.product_name,
    SUM(o.quantity) AS total_sold,
    ROW_NUMBER() OVER (ORDER BY SUM(o.quantity) DESC) AS row_number_product
FROM
    order_details o
JOIN
    products p ON p.product_id = o.product_id
GROUP BY
    p.product_name; 
''' 
pd.read_sql(query, conn)

,product_name,total_sold,row_number_product
0,Camembert Pierrot,1577,1
1,Raclette Courdavault,1496,2
2,Gorgonzola Telino,1397,3
3,Gnocchi di nonna Alice,1263,4
4,Pavlova,1158,5
...,...,...,...
72,Laughing Lumberjack Lager,184,73
73,Chocolade,138,74
74,Gravad lax,125,75
75,Genen Shouyu,122,76


### Explicação:
* RANK(): Atribui um ranking aos produtos, mas pode haver laços. Se dois produtos tiverem a mesma quantidade vendida, eles receberão o mesmo ranking.
* DENSE_RANK(): Também atribui um ranking, mas não pula números. Se houver laços, o próximo ranking será atribuído ao próximo produto, mesmo que ele tenha a mesma quantidade vendida.
* ROW_NUMBER(): Atribui um número sequencial exclusivo a cada linha, mesmo que haja laços.
### Duas Implementações:
* Com Subquery: A subquery é utilizada para calcular o total vendido de cada produto primeiro. A partir disso, a função de ranking é aplicada.
* Sem Subquery: A função de ranking é aplicada diretamente ao resultado da agregação do total vendido.

* Listar funcionários dividindo-os em 3 grupos usando NTILE
*  FROM employees;

In [29]:
query = ''' 
SELECT
    employee_id,
    first_name,
    last_name,
    NTILE(3) OVER (ORDER BY employee_id) AS employee_group
FROM
    employees;
'''
pd.read_sql(query, conn)

,employee_id,first_name,last_name,employee_group
0,1,Nancy,Davolio,1
1,2,Andrew,Fuller,1
2,3,Janet,Leverling,1
3,4,Margaret,Peacock,2
4,5,Steven,Buchanan,2
5,6,Michael,Suyama,2
6,7,Robert,King,3
7,8,Laura,Callahan,3
8,9,Anne,Dodsworth,3


### Explicação:
* NTILE(3): Divide os funcionários em 3 grupos iguais (ou o mais próximo possível de iguais) com base no employee_id.
* ORDER BY employee_id: Ordena os funcionários por employee_id antes da divisão.
* employee_group: Atribui um número de grupo a cada funcionário (1, 2 ou 3).

In [ ]:
*  Ordenando os custos de envio pagos pelos clientes de acordo com suas datas de pedido, mostrando o custo anterior e o custo posterior usando LAG e LEAD:
-- FROM orders JOIN shippers ON shippers.shipper_id = orders.ship_via;

In [33]:
query = ''' 
SELECT
    o.order_id,
    o.order_date,
    s.company_name,
    o.freight AS current_freight,
    LAG(o.freight, 1, 0) OVER (ORDER BY o.order_date) AS previous_freight,
    LEAD(o.freight, 1, 0) OVER (ORDER BY o.order_date) AS next_freight
FROM
    orders o
JOIN
    shippers s ON s.shipper_id = o.ship_via
ORDER BY
    o.order_date;
'''
pd.read_sql(query, conn)

,order_id,order_date,company_name,current_freight,previous_freight,next_freight
0,10248,1996-07-04,Federal Shipping,32.38,0.00,11.61
1,10249,1996-07-05,Speedy Express,11.61,32.38,65.83
2,10250,1996-07-08,United Package,65.83,11.61,41.34
3,10251,1996-07-08,Speedy Express,41.34,65.83,51.30
4,10252,1996-07-09,United Package,51.30,41.34,58.17
...,...,...,...,...,...,...
825,11073,1998-05-05,United Package,24.95,258.64,18.44
826,11074,1998-05-06,United Package,18.44,24.95,6.19
827,11075,1998-05-06,United Package,6.19,18.44,38.28
828,11076,1998-05-06,United Package,38.28,6.19,8.53


### Explicação:
* LAG(o.freight, 1, 0) OVER (ORDER BY o.order_date): Obtém o custo de frete do pedido anterior à linha atual, ordenado por order_date. O segundo argumento (1) indica que queremos o valor anterior (offset de 1 linha). O terceiro argumento (0) define o valor padrão caso não haja um pedido anterior.
* LEAD(o.freight, 1, 0) OVER (ORDER BY o.order_date): Obtém o custo de frete do pedido seguinte à linha atual, ordenado por order_date.
* ORDER BY o.order_date: Ordena os resultados por data do pedido para que as funções LAG e LEAD funcionem corretamente.

-- Faça a classificação dos produtos mais venvidos usando usando RANK(), DENSE_RANK() e ROW_NUMBER()
-- Essa questão tem 2 implementações, veja uma que utiliza subquery e uma que não utiliza.
-- Tabelas utilizadasFROM order_details o JOIN products p ON p.product_id = o.product_id;

-- Listar funcionários dividindo-os em 3 grupos usando NTILE
-- FROM employees;

-- Ordenando os custos de envio pagos pelos clientes de acordo com suas datas de pedido, mostrando o custo anterior e o custo posterior usando LAG e LEAD:
-- FROM orders JOIN shippers ON shippers.shipper_id = orders.ship_via;

-- Desafio extra: questão intrevista Google
-- https://medium.com/@aggarwalakshima/interview-question-asked-by-google-and-difference-among-row-number-rank-and-dense-rank-4ca08f888486#:~:text=ROW_NUMBER()%20always%20provides%20unique,a%20continuous%20sequence%20of%20ranks.
-- https://platform.stratascratch.com/coding/10351-activity-rank?code_type=3
-- https://www.youtube.com/watch?v=db-qdlp8u3o